In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
from datetime import datetime
import scipy.stats as stats

start = datetime.now()
df = pd.read_csv("C:\\Users\\Jens Harbers\\Documents\\schedenveg_reproduce.csv",sep=" ")
items = df.copy()
# manually remove some species, might be a different approach for rule mining in graslands
n = df.area.nunique()
cnt = df.variable.value_counts()/n

df = df[~df.variable.isin(cnt[(cnt < 0.12) | (cnt >= 0.8)].index.values.tolist())]
df = pd.crosstab(df.area,df.variable)
frequent_itemsets = fpgrowth(df, min_support=0.12, use_colnames=True, max_len=20)

assoc = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.05, support_only=False)

stop = datetime.now()



assoc['n1x'] = assoc['antecedent support'] * n
assoc['nx1'] = assoc['consequent support'] * n
assoc['n11'] = assoc['support'] * n

assoc['n0x'] = n - assoc['n1x']
assoc['nx0'] = n - assoc['nx1']

assoc['n10'] = assoc["n1x"] - assoc['n11']
assoc['n01'] = assoc["nx1"] - assoc['n11']
assoc['n00'] = assoc['n0x'] - assoc['n01'] 

# assoc


# assoc.to_parquet("assoc_rules_schedenveg.parquet",index=False)
species_on_field = items[items.area == "A10_16"].variable.values

# true if all sciecies in antecedents are also existent in field
assoc["antecedent_in_field"] = assoc.apply(lambda x: x.antecedents.issubset(species_on_field), axis=1)
assoc = assoc[assoc.antecedent_in_field==True]
# True is no species is listed in consequent
assoc["consequent_not_in_field"] = assoc.apply(lambda x: x.consequents.isdisjoint(species_on_field), axis=1)
assoc = assoc[assoc.consequent_not_in_field==True]

print(stop - start)